In [1]:
import pandas as pd
import numpy as np

df_annual_em_science_2019 = pd.read_csv(
    "/Users/kashafali/Documents/Duke/Spring23/UDS/Project/Data/2019/Annual EM Science.csv"
)


In [2]:
df_annual_em_science_2019.columns


Index(['ENTITY_CD', 'ENTITY_NAME', 'YEAR', 'ASSESSMENT_NAME', 'SUBGROUP_NAME',
       'NUM_TESTED', 'NOT_TESTED', 'LEVEL1_COUNT', 'LEVEL1_%TESTED',
       'LEVEL2_COUNT', 'LEVEL2_%TESTED', 'LEVEL3_COUNT', 'LEVEL3_%TESTED',
       'LEVEL4_COUNT', 'LEVEL4_%TESTED', 'NUM_PROF', 'PER_PROF',
       'TOTAL_SCALE_SCORES', 'MEAN_SCORE'],
      dtype='object')

In [3]:
df_annual_em_science_2019["SUBGROUP_NAME"].unique()

array(['All Students', 'American Indian or Alaska Native',
       'Asian or Native Hawaiian/Other Pacific Islander',
       'Black or African American', 'Economically Disadvantaged',
       'English Language Learners', 'Female', 'General Education',
       'Hispanic or Latino', 'Homeless', 'Male', 'Multiracial',
       'Non-English Language Learners', 'Not Economically Disadvantaged',
       'Not Homeless', 'Not in Foster Care', 'Not Migrant',
       'Parent in Armed Forces', 'Parent Not in Armed Forces',
       'Students with Disabilities', 'White', 'In Foster Care', 'Migrant',
       'Small Group Total'], dtype=object)

In [4]:
# rename Non-English Language Learners to Non-English Language Learner
df_annual_em_science_2019["SUBGROUP_NAME"] = df_annual_em_science_2019["SUBGROUP_NAME"].replace("Non-English Language Learners", "Non-English Language Learner")
# rename English Language Learners to English Language Learner
df_annual_em_science_2019["SUBGROUP_NAME"] = df_annual_em_science_2019["SUBGROUP_NAME"].replace("English Language Learners", "English Language Learner")


# Dataframe for All Students

In [5]:
# only keep columns
column_names = [
    "ENTITY_CD",
    "ENTITY_NAME",
    "YEAR",
    "ASSESSMENT_NAME",
    "SUBGROUP_NAME",
    "NOT_TESTED",
    "NUM_TESTED",
    "NUM_PROF",
    "PER_PROF",
    "TOTAL_SCALE_SCORES",
    "MEAN_SCORE",
]

df_annual_em_science_2019_total = df_annual_em_science_2019[column_names]
df_annual_em_science_2019_total = df_annual_em_science_2019_total[
    df_annual_em_science_2019_total["SUBGROUP_NAME"] == "All Students"
]


In [6]:
df_annual_em_science_2019_total.head()


,ENTITY_CD,ENTITY_NAME,YEAR,ASSESSMENT_NAME,SUBGROUP_NAME,NOT_TESTED,NUM_TESTED,NUM_PROF,PER_PROF,TOTAL_SCALE_SCORES,MEAN_SCORE
0,1,NYC Public Schools,2018,Science4,All Students,8549,68089,59021,87,5510492,81
1,1,NYC Public Schools,2018,Science8,All Students,23690,47519,25375,53,3062957,64
2,1,NYC Public Schools,2018,RegentsScience8,All Students,s,14463,11338,78,NaN,NaN
3,1,NYC Public Schools,2018,CombinedScience,All Students,23690,61982,36713,59,3062957,49
4,1,NYC Public Schools,2018,Science4_8,All Students,32239,130071,95734,74,8573449,66


In [7]:
df_annual_em_science_2019_merge = df_annual_em_science_2019_total.drop(
    columns=["SUBGROUP_NAME"]
).copy()


# Dataframe for Gender

In [8]:
df_annual_em_science_2019_gender = df_annual_em_science_2019.copy()

df_annual_em_science_2019_gender = df_annual_em_science_2019_gender[column_names]

df_annual_em_science_2019_gender = df_annual_em_science_2019_gender[
    (df_annual_em_science_2019_gender["SUBGROUP_NAME"] == "Female")
    | (df_annual_em_science_2019_gender["SUBGROUP_NAME"] == "Male")
]


In [9]:
# fix index
df_annual_em_science_2019_gender = df_annual_em_science_2019_gender.set_index(
    ["SUBGROUP_NAME", "ENTITY_CD", "YEAR", "ASSESSMENT_NAME"]
).unstack(["SUBGROUP_NAME"])

df_annual_em_science_2019_gender = df_annual_em_science_2019_gender.reindex(
    columns=sorted(df_annual_em_science_2019_gender.columns, key=lambda x: x[::-1])
)

df_annual_em_science_2019_gender.columns = [
    "{}_{}".format(t, v) for v, t in df_annual_em_science_2019_gender.columns
]

df_annual_em_science_2019_gender = pd.DataFrame(df_annual_em_science_2019_gender.to_records())


In [10]:
df_annual_em_science_2019_gender.head()


,ENTITY_CD,YEAR,ASSESSMENT_NAME,Female_ENTITY_NAME,Female_MEAN_SCORE,Female_NOT_TESTED,Female_NUM_PROF,Female_NUM_TESTED,Female_PER_PROF,Female_TOTAL_SCALE_SCORES,Male_ENTITY_NAME,Male_MEAN_SCORE,Male_NOT_TESTED,Male_NUM_PROF,Male_NUM_TESTED,Male_PER_PROF,Male_TOTAL_SCALE_SCORES
0,1,2018,RegentsScience8,NYC Public Schools,NaN,s,6137,7685.0,80,NaN,NYC Public Schools,NaN,s,5201,6778.0,77,NaN
1,1,2018,Science4,NYC Public Schools,81,3452,29153,33312.0,88,2705318,NYC Public Schools,81,5097,29868,34777.0,86,2805174
2,1,2018,Science8,NYC Public Schools,65,11546,12452,22638.0,55,1479308,NYC Public Schools,64,12144,12923,24881.0,52,1583649
3,1,2019,RegentsScience8,NYC Public Schools,NaN,s,6515,8374.0,78,NaN,NYC Public Schools,NaN,s,5568,7360.0,76,NaN
4,1,2019,Science4,NYC Public Schools,80,2995,28158,33190.0,85,2642687,NYC Public Schools,78,4852,28830,34897.0,83,2737289


# Merge Gender Dataframe with All Students Dataframe

In [11]:
df_science_2019_1 = pd.merge(
    df_annual_em_science_2019_merge,
    df_annual_em_science_2019_gender,
    on=["ENTITY_CD", "YEAR", "ASSESSMENT_NAME"],
    how="left",
)

df_science_2019_1.head()


,ENTITY_CD,ENTITY_NAME,YEAR,ASSESSMENT_NAME,NOT_TESTED,NUM_TESTED,NUM_PROF,PER_PROF,TOTAL_SCALE_SCORES,MEAN_SCORE,...,Female_NUM_TESTED,Female_PER_PROF,Female_TOTAL_SCALE_SCORES,Male_ENTITY_NAME,Male_MEAN_SCORE,Male_NOT_TESTED,Male_NUM_PROF,Male_NUM_TESTED,Male_PER_PROF,Male_TOTAL_SCALE_SCORES
0,1,NYC Public Schools,2018,Science4,8549,68089,59021,87,5510492,81,...,33312.0,88,2705318,NYC Public Schools,81,5097,29868,34777.0,86,2805174
1,1,NYC Public Schools,2018,Science8,23690,47519,25375,53,3062957,64,...,22638.0,55,1479308,NYC Public Schools,64,12144,12923,24881.0,52,1583649
2,1,NYC Public Schools,2018,RegentsScience8,s,14463,11338,78,NaN,NaN,...,7685.0,80,NaN,NYC Public Schools,NaN,s,5201,6778.0,77,NaN
3,1,NYC Public Schools,2018,CombinedScience,23690,61982,36713,59,3062957,49,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,NYC Public Schools,2018,Science4_8,32239,130071,95734,74,8573449,66,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Dataframe for Race

In [12]:
df_annual_em_science_2019_race = df_annual_em_science_2019[column_names].copy()

df_annual_em_science_2019_race = df_annual_em_science_2019_race[
    (df_annual_em_science_2019_race["SUBGROUP_NAME"] == "American Indian or Alaska Native")
    | (df_annual_em_science_2019_race["SUBGROUP_NAME"] == "Black or African American")
    | (df_annual_em_science_2019_race["SUBGROUP_NAME"] == "Hispanic or Latino")
    | (
        df_annual_em_science_2019_race["SUBGROUP_NAME"]
        == "Asian or Native Hawaiian/Other Pacific Islander"
    )
    | (df_annual_em_science_2019_race["SUBGROUP_NAME"] == "White")
    | (df_annual_em_science_2019_race["SUBGROUP_NAME"] == "Multiracial")
]


In [13]:
# reindex

df_annual_em_science_2019_race = df_annual_em_science_2019_race.set_index(
    ["SUBGROUP_NAME", "ENTITY_CD", "YEAR", "ASSESSMENT_NAME"]
).unstack(["SUBGROUP_NAME"])

df_annual_em_science_2019_race = df_annual_em_science_2019_race.reindex(
    columns=sorted(df_annual_em_science_2019_race.columns, key=lambda x: x[::-1])
)

df_annual_em_science_2019_race.columns = [
    "{}_{}".format(t, v) for v, t in df_annual_em_science_2019_race.columns
]

df_annual_em_science_2019_race = pd.DataFrame(df_annual_em_science_2019_race.to_records())


In [14]:
df_annual_em_science_2019_race.head()


,ENTITY_CD,YEAR,ASSESSMENT_NAME,American Indian or Alaska Native_ENTITY_NAME,American Indian or Alaska Native_MEAN_SCORE,American Indian or Alaska Native_NOT_TESTED,American Indian or Alaska Native_NUM_PROF,American Indian or Alaska Native_NUM_TESTED,American Indian or Alaska Native_PER_PROF,American Indian or Alaska Native_TOTAL_SCALE_SCORES,...,Multiracial_NUM_TESTED,Multiracial_PER_PROF,Multiracial_TOTAL_SCALE_SCORES,White_ENTITY_NAME,White_MEAN_SCORE,White_NOT_TESTED,White_NUM_PROF,White_NUM_TESTED,White_PER_PROF,White_TOTAL_SCALE_SCORES
0,1,2018,RegentsScience8,NYC Public Schools,NaN,s,121,178.0,68,NaN,...,219.0,95,NaN,NYC Public Schools,NaN,s,2771,2986.0,93,NaN
1,1,2018,Science4,NYC Public Schools,80,62,629,711.0,88,57233,...,835.0,96,73692,NYC Public Schools,87,2900,10606,11228.0,94,974616
2,1,2018,Science8,NYC Public Schools,62,286,253,528.0,48,32605,...,259.0,71,18781,NYC Public Schools,72,5461,5130,7035.0,73,508640
3,1,2019,RegentsScience8,NYC Public Schools,NaN,s,144,204.0,71,NaN,...,280.0,90,NaN,NYC Public Schools,NaN,s,3003,3243.0,93,NaN
4,1,2019,Science4,NYC Public Schools,78,59,617,743.0,83,58136,...,959.0,94,82681,NYC Public Schools,85,2670,10622,11431.0,93,973658


# Merge Race Dataset with All Students + Gender Dataset

In [15]:
df_science_2019_2 = pd.merge(
    df_science_2019_1,
    df_annual_em_science_2019_race,
    on=["ENTITY_CD", "YEAR", "ASSESSMENT_NAME"],
    how="left",
)

df_science_2019_2.head()


,ENTITY_CD,ENTITY_NAME,YEAR,ASSESSMENT_NAME,NOT_TESTED,NUM_TESTED,NUM_PROF,PER_PROF,TOTAL_SCALE_SCORES,MEAN_SCORE,...,Multiracial_NUM_TESTED,Multiracial_PER_PROF,Multiracial_TOTAL_SCALE_SCORES,White_ENTITY_NAME,White_MEAN_SCORE,White_NOT_TESTED,White_NUM_PROF,White_NUM_TESTED,White_PER_PROF,White_TOTAL_SCALE_SCORES
0,1,NYC Public Schools,2018,Science4,8549,68089,59021,87,5510492,81,...,835.0,96,73692,NYC Public Schools,87,2900,10606,11228.0,94,974616
1,1,NYC Public Schools,2018,Science8,23690,47519,25375,53,3062957,64,...,259.0,71,18781,NYC Public Schools,72,5461,5130,7035.0,73,508640
2,1,NYC Public Schools,2018,RegentsScience8,s,14463,11338,78,NaN,NaN,...,219.0,95,NaN,NYC Public Schools,NaN,s,2771,2986.0,93,NaN
3,1,NYC Public Schools,2018,CombinedScience,23690,61982,36713,59,3062957,49,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,NYC Public Schools,2018,Science4_8,32239,130071,95734,74,8573449,66,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# English Language Learners Dataframe

In [16]:
df_annual_em_science_2019_eng = df_annual_em_science_2019[column_names].copy()


df_annual_em_science_2019_eng = df_annual_em_science_2019_eng[
    (df_annual_em_science_2019_eng["SUBGROUP_NAME"] == "Non-English Language Learner")
    | (df_annual_em_science_2019_eng["SUBGROUP_NAME"] == "English Language Learner")
]


In [17]:
# reindex

df_annual_em_science_2019_eng = df_annual_em_science_2019_eng.set_index(
    ["SUBGROUP_NAME", "ENTITY_CD", "YEAR", "ASSESSMENT_NAME"]
).unstack(["SUBGROUP_NAME"])

df_annual_em_science_2019_eng = df_annual_em_science_2019_eng.reindex(
    columns=sorted(df_annual_em_science_2019_eng.columns, key=lambda x: x[::-1])
)

df_annual_em_science_2019_eng.columns = [
    "{}_{}".format(t, v) for v, t in df_annual_em_science_2019_eng.columns
]

df_annual_em_science_2019_eng = pd.DataFrame(df_annual_em_science_2019_eng.to_records())


In [18]:
df_annual_em_science_2019_eng.head()


,ENTITY_CD,YEAR,ASSESSMENT_NAME,English Language Learner_ENTITY_NAME,English Language Learner_MEAN_SCORE,English Language Learner_NOT_TESTED,English Language Learner_NUM_PROF,English Language Learner_NUM_TESTED,English Language Learner_PER_PROF,English Language Learner_TOTAL_SCALE_SCORES,Non-English Language Learner_ENTITY_NAME,Non-English Language Learner_MEAN_SCORE,Non-English Language Learner_NOT_TESTED,Non-English Language Learner_NUM_PROF,Non-English Language Learner_NUM_TESTED,Non-English Language Learner_PER_PROF,Non-English Language Learner_TOTAL_SCALE_SCORES
0,1,2018,RegentsScience8,NYC Public Schools,NaN,s,95,507.0,19,NaN,NYC Public Schools,NaN,s,11243,13956.0,81,NaN
1,1,2018,Science4,NYC Public Schools,69,1113,7266,11102.0,65,761317,NYC Public Schools,83,7436,51755,56987.0,91,4749175
2,1,2018,Science8,NYC Public Schools,49,1784,1309,7054.0,19,346288,NYC Public Schools,67,21906,24066,40465.0,59,2716669
3,1,2019,RegentsScience8,NYC Public Schools,NaN,s,101,547.0,18,NaN,NYC Public Schools,NaN,s,11982,15187.0,79,NaN
4,1,2019,Science4,NYC Public Schools,66,1078,6436,11006.0,58,720965,NYC Public Schools,82,6769,50552,57081.0,89,4659011


# Merge English Language Learners Dataset with All Students + Gender + Race Dataset

In [19]:
df_science_2019_3 = pd.merge(
    df_science_2019_2,
    df_annual_em_science_2019_eng,
    on=["ENTITY_CD", "YEAR", "ASSESSMENT_NAME"],
    how="left",
)

df_science_2019_3.head()


,ENTITY_CD,ENTITY_NAME,YEAR,ASSESSMENT_NAME,NOT_TESTED,NUM_TESTED,NUM_PROF,PER_PROF,TOTAL_SCALE_SCORES,MEAN_SCORE,...,English Language Learner_NUM_TESTED,English Language Learner_PER_PROF,English Language Learner_TOTAL_SCALE_SCORES,Non-English Language Learner_ENTITY_NAME,Non-English Language Learner_MEAN_SCORE,Non-English Language Learner_NOT_TESTED,Non-English Language Learner_NUM_PROF,Non-English Language Learner_NUM_TESTED,Non-English Language Learner_PER_PROF,Non-English Language Learner_TOTAL_SCALE_SCORES
0,1,NYC Public Schools,2018,Science4,8549,68089,59021,87,5510492,81,...,11102.0,65,761317,NYC Public Schools,83,7436,51755,56987.0,91,4749175
1,1,NYC Public Schools,2018,Science8,23690,47519,25375,53,3062957,64,...,7054.0,19,346288,NYC Public Schools,67,21906,24066,40465.0,59,2716669
2,1,NYC Public Schools,2018,RegentsScience8,s,14463,11338,78,NaN,NaN,...,507.0,19,NaN,NYC Public Schools,NaN,s,11243,13956.0,81,NaN
3,1,NYC Public Schools,2018,CombinedScience,23690,61982,36713,59,3062957,49,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,NYC Public Schools,2018,Science4_8,32239,130071,95734,74,8573449,66,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Dataframe for Economic Condition

In [20]:
df_annual_em_science_2019_econ = df_annual_em_science_2019[column_names].copy()

df_annual_em_science_2019_econ = df_annual_em_science_2019_econ[
    (df_annual_em_science_2019_econ["SUBGROUP_NAME"] == "Economically Disadvantaged")
    | (df_annual_em_science_2019_econ["SUBGROUP_NAME"] == "Not Economically Disadvantaged")
]


In [21]:
# reindex

df_annual_em_science_2019_econ = df_annual_em_science_2019_econ.set_index(
    ["SUBGROUP_NAME", "ENTITY_CD", "YEAR", "ASSESSMENT_NAME"]
).unstack(["SUBGROUP_NAME"])

df_annual_em_science_2019_econ = df_annual_em_science_2019_econ.reindex(
    columns=sorted(df_annual_em_science_2019_econ.columns, key=lambda x: x[::-1])
)

df_annual_em_science_2019_econ.columns = [
    "{}_{}".format(t, v) for v, t in df_annual_em_science_2019_econ.columns
]

df_annual_em_science_2019_econ = pd.DataFrame(df_annual_em_science_2019_econ.to_records())


In [22]:
df_annual_em_science_2019_econ.head()


,ENTITY_CD,YEAR,ASSESSMENT_NAME,Economically Disadvantaged_ENTITY_NAME,Economically Disadvantaged_MEAN_SCORE,Economically Disadvantaged_NOT_TESTED,Economically Disadvantaged_NUM_PROF,Economically Disadvantaged_NUM_TESTED,Economically Disadvantaged_PER_PROF,Economically Disadvantaged_TOTAL_SCALE_SCORES,Not Economically Disadvantaged_ENTITY_NAME,Not Economically Disadvantaged_MEAN_SCORE,Not Economically Disadvantaged_NOT_TESTED,Not Economically Disadvantaged_NUM_PROF,Not Economically Disadvantaged_NUM_TESTED,Not Economically Disadvantaged_PER_PROF,Not Economically Disadvantaged_TOTAL_SCALE_SCORES
0,1,2018,RegentsScience8,NYC Public Schools,NaN,s,6409,8934.0,72,NaN,NYC Public Schools,NaN,s,4929,5529.0,89,NaN
1,1,2018,Science4,NYC Public Schools,79,4405,43241,51467.0,84,4058913,NYC Public Schools,87,4144,15780,16622.0,95,1451579
2,1,2018,Science8,NYC Public Schools,63,14468,18515,37612.0,49,2355289,NYC Public Schools,71,9222,6860,9907.0,69,707668
3,1,2019,RegentsScience8,NYC Public Schools,NaN,s,6663,9576.0,70,NaN,NYC Public Schools,NaN,s,5420,6158.0,88,NaN
4,1,2019,Science4,NYC Public Schools,77,3844,40390,50271.0,80,3857541,NYC Public Schools,85,4003,16598,17816.0,93,1522435


# Merge Economic Dataset with All Students + Gender + Race + English Language Dataset

In [23]:
df_science_2019_4 = pd.merge(
    df_science_2019_3,
    df_annual_em_science_2019_econ,
    on=["ENTITY_CD", "YEAR", "ASSESSMENT_NAME"],
    how="left",
)

df_science_2019_4.head()


,ENTITY_CD,ENTITY_NAME,YEAR,ASSESSMENT_NAME,NOT_TESTED,NUM_TESTED,NUM_PROF,PER_PROF,TOTAL_SCALE_SCORES,MEAN_SCORE,...,Economically Disadvantaged_NUM_TESTED,Economically Disadvantaged_PER_PROF,Economically Disadvantaged_TOTAL_SCALE_SCORES,Not Economically Disadvantaged_ENTITY_NAME,Not Economically Disadvantaged_MEAN_SCORE,Not Economically Disadvantaged_NOT_TESTED,Not Economically Disadvantaged_NUM_PROF,Not Economically Disadvantaged_NUM_TESTED,Not Economically Disadvantaged_PER_PROF,Not Economically Disadvantaged_TOTAL_SCALE_SCORES
0,1,NYC Public Schools,2018,Science4,8549,68089,59021,87,5510492,81,...,51467.0,84,4058913,NYC Public Schools,87,4144,15780,16622.0,95,1451579
1,1,NYC Public Schools,2018,Science8,23690,47519,25375,53,3062957,64,...,37612.0,49,2355289,NYC Public Schools,71,9222,6860,9907.0,69,707668
2,1,NYC Public Schools,2018,RegentsScience8,s,14463,11338,78,NaN,NaN,...,8934.0,72,NaN,NYC Public Schools,NaN,s,4929,5529.0,89,NaN
3,1,NYC Public Schools,2018,CombinedScience,23690,61982,36713,59,3062957,49,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,NYC Public Schools,2018,Science4_8,32239,130071,95734,74,8573449,66,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Dataframe for Special Education

In [24]:
df_annual_em_science_2019_dis = df_annual_em_science_2019[column_names].copy()

df_annual_em_science_2019_dis = df_annual_em_science_2019_dis[
    (df_annual_em_science_2019_dis["SUBGROUP_NAME"] == "Students with Disabilities")
]


In [25]:
# reindex

df_annual_em_science_2019_dis = df_annual_em_science_2019_dis.set_index(
    ["SUBGROUP_NAME", "ENTITY_CD", "YEAR", "ASSESSMENT_NAME"]
).unstack(["SUBGROUP_NAME"])

df_annual_em_science_2019_dis = df_annual_em_science_2019_dis.reindex(
    columns=sorted(df_annual_em_science_2019_dis.columns, key=lambda x: x[::-1])
)

df_annual_em_science_2019_dis.columns = [
    "{}_{}".format(t, v) for v, t in df_annual_em_science_2019_dis.columns
]

df_annual_em_science_2019_dis = pd.DataFrame(df_annual_em_science_2019_dis.to_records())


In [26]:
df_annual_em_science_2019_dis.head()


,ENTITY_CD,YEAR,ASSESSMENT_NAME,Students with Disabilities_ENTITY_NAME,Students with Disabilities_MEAN_SCORE,Students with Disabilities_NOT_TESTED,Students with Disabilities_NUM_PROF,Students with Disabilities_NUM_TESTED,Students with Disabilities_PER_PROF,Students with Disabilities_TOTAL_SCALE_SCORES
0,1,2018,RegentsScience8,NYC Public Schools,NaN,s,537,1328,40,NaN
1,1,2018,Science4,NYC Public Schools,74,6412,11052,14793,75,1087759
2,1,2018,Science8,NYC Public Schools,54,7136,2910,10589,27,567837
3,1,2019,RegentsScience8,NYC Public Schools,NaN,s,669,1586,42,NaN
4,1,2019,Science4,NYC Public Schools,71,6324,10422,15221,68,1073585


# Merge Special Education Dataset with All Students + Gender + Race + English Language + Econ Dataset

In [27]:
df_science_2019_5 = pd.merge(
    df_science_2019_4,
    df_annual_em_science_2019_dis,
    on=["ENTITY_CD", "YEAR", "ASSESSMENT_NAME"],
    how="left",
)

df_science_2019_5.head()


,ENTITY_CD,ENTITY_NAME,YEAR,ASSESSMENT_NAME,NOT_TESTED,NUM_TESTED,NUM_PROF,PER_PROF,TOTAL_SCALE_SCORES,MEAN_SCORE,...,Not Economically Disadvantaged_NUM_TESTED,Not Economically Disadvantaged_PER_PROF,Not Economically Disadvantaged_TOTAL_SCALE_SCORES,Students with Disabilities_ENTITY_NAME,Students with Disabilities_MEAN_SCORE,Students with Disabilities_NOT_TESTED,Students with Disabilities_NUM_PROF,Students with Disabilities_NUM_TESTED,Students with Disabilities_PER_PROF,Students with Disabilities_TOTAL_SCALE_SCORES
0,1,NYC Public Schools,2018,Science4,8549,68089,59021,87,5510492,81,...,16622.0,95,1451579,NYC Public Schools,74,6412,11052,14793.0,75,1087759
1,1,NYC Public Schools,2018,Science8,23690,47519,25375,53,3062957,64,...,9907.0,69,707668,NYC Public Schools,54,7136,2910,10589.0,27,567837
2,1,NYC Public Schools,2018,RegentsScience8,s,14463,11338,78,NaN,NaN,...,5529.0,89,NaN,NYC Public Schools,NaN,s,537,1328.0,40,NaN
3,1,NYC Public Schools,2018,CombinedScience,23690,61982,36713,59,3062957,49,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,NYC Public Schools,2018,Science4_8,32239,130071,95734,74,8573449,66,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Clean final ELA dataset

In [28]:
# show all columns in dataframe

# Drop Entity Name columns

df_science_2019_5 = df_science_2019_5.drop(
    columns=['Female_ENTITY_NAME', 
    'Male_ENTITY_NAME',
    'American Indian or Alaska Native_ENTITY_NAME',
    'Black or African American_ENTITY_NAME',
    'Hispanic or Latino_ENTITY_NAME',
    'Asian or Native Hawaiian/Other Pacific Islander_ENTITY_NAME',
    'White_ENTITY_NAME',
    'Multiracial_ENTITY_NAME',
    'Non-English Language Learner_ENTITY_NAME',
    'English Language Learner_ENTITY_NAME',
    'Economically Disadvantaged_ENTITY_NAME',
    'Not Economically Disadvantaged_ENTITY_NAME',
    'Students with Disabilities_ENTITY_NAME']
)



In [29]:
df_science_2019_5.head()

,ENTITY_CD,ENTITY_NAME,YEAR,ASSESSMENT_NAME,NOT_TESTED,NUM_TESTED,NUM_PROF,PER_PROF,TOTAL_SCALE_SCORES,MEAN_SCORE,...,Not Economically Disadvantaged_NUM_PROF,Not Economically Disadvantaged_NUM_TESTED,Not Economically Disadvantaged_PER_PROF,Not Economically Disadvantaged_TOTAL_SCALE_SCORES,Students with Disabilities_MEAN_SCORE,Students with Disabilities_NOT_TESTED,Students with Disabilities_NUM_PROF,Students with Disabilities_NUM_TESTED,Students with Disabilities_PER_PROF,Students with Disabilities_TOTAL_SCALE_SCORES
0,1,NYC Public Schools,2018,Science4,8549,68089,59021,87,5510492,81,...,15780,16622.0,95,1451579,74,6412,11052,14793.0,75,1087759
1,1,NYC Public Schools,2018,Science8,23690,47519,25375,53,3062957,64,...,6860,9907.0,69,707668,54,7136,2910,10589.0,27,567837
2,1,NYC Public Schools,2018,RegentsScience8,s,14463,11338,78,NaN,NaN,...,4929,5529.0,89,NaN,NaN,s,537,1328.0,40,NaN
3,1,NYC Public Schools,2018,CombinedScience,23690,61982,36713,59,3062957,49,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,NYC Public Schools,2018,Science4_8,32239,130071,95734,74,8573449,66,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
# save to csv

df_science_2019_5.to_csv('df_science_2019.csv', index=False)